# Calculating Scheduled Headways for CTA Buses - work in progress

### What This Does

TBD

## In progress

- Calculate scheduled headways at each stop, making sure they make sense given different patterns that
run at different times of day for each route


## To Do Next

TBD

## Notes

TBD


## Strategy for Scheduled Headways (Tentative)

1. Which service IDs serve a given stop?

2. When are the earliest/latest times those service IDs are in effect?

3. What are the headways within each timeframe?

## Needs Fixing
 
TBD

In [1]:
import requests
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
import datetime as dt
import numpy as np
from static_gtfs_analysis import *
from 

In [2]:
# # Get API key from the .env file
# load_dotenv()
# API_KEY = os.getenv('API_KEY')

# Get Scheduled Stop Times and Headways


In [3]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format('20230105')

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.56it/s]


In [2]:
%%capture --no-display
# turn off warnings

######### Scratch
gtfs_version_id = '20230105'
route_id = '55'
service_date_string = '2023-01-09'
# start_timedelta_string ='02:30'
# end_timedelta_string = '26:30'


In [3]:
def string_to_datetime(date_string):
        year = int(date_string[:4])
        month = int(date_string[5:7])
        day = int(date_string[8:])
        return pendulum.datetime(year, month, day)


In [4]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format(gtfs_version_id)

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:05<00:00,  1.35it/s]


In [9]:
def get_stop_details(gtfs_feed, route_id, service_date_string):

    service_date = string_to_datetime(service_date_string)

    # Get trip summary for service date using chn-ghost-buses make_trip_summary() function
    trip_summary = make_trip_summary(gtfs_feed, service_date, service_date)

    # filter down to the specified route
    trip_summary = trip_summary[trip_summary['route_id'] == route_id]

    # list trip ids for this route
    trip_list = trip_summary['trip_id'].unique().tolist()

    # get stop times data for the trips on this route
    stop_times = gtfs_feed.stop_times

    # filter stop times down to the relevant trips
    stop_times = stop_times.loc[stop_times['trip_id'].isin(trip_list)]

    # Add service id, route, and direction to the stop times data
    stop_times = stop_times.merge(trip_summary[['trip_id', 'route_id', 'service_id', 'direction']], on='trip_id')

    # filter stop details down to the relevant route
    stop_times = stop_times.loc[stop_times['route_id'] == route_id]

    # Eliminate duplicates - every line shows up twice.
    # TODO:  Investigate why. Is this related to the calendar_cross line in the 
    # make_trip_summary() function? And/or the fact that I'm using the 
    # same date as start and end date as arguments in make_trip_summary?()
    stop_times = stop_times.drop_duplicates()

    return stop_times

stop_details = get_stop_details(gtfs_feed, route_id, service_date_string)
stop_details

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction
0,6530000010020,04:37:30,04:37:30,10603,1,St Louis,0,0,4,4,55,65301,West
1,6530000010020,04:37:55,04:37:55,10604,2,St Louis,0,639,4,4,55,65301,West
2,6530000010020,04:38:31,04:38:31,10605,3,St Louis,0,1375,4,4,55,65301,West
3,6530000010020,04:39:01,04:39:01,10606,4,St Louis,0,2061,4,4,55,65301,West
4,6530000010020,04:39:32,04:39:32,10607,5,St Louis,0,2826,4,4,55,65301,West
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26368,6530045537030,23:38:09,23:38:09,15366,63,Midway Orange Line,0,45089,23,23,55,65302,West
26369,6530045537030,23:38:42,23:38:42,10632,64,Midway Orange Line,0,45760,23,23,55,65302,West
26370,6530045537030,23:39:14,23:39:14,10633,65,Midway Orange Line,0,46457,23,23,55,65302,West
26371,6530045537030,23:39:44,23:39:44,17030,66,Midway Orange Line,0,47108,23,23,55,65302,West


In [33]:
# Get headways
def get_headways(stop_details):
    
    df_headways = pd.DataFrame()

    # consider each stop and direction of travel separately
    for stop_id, direction in set(zip(stop_details['stop_id'], stop_details['direction'])):

        # dataframe filtered to one stop_id and direction
        df = stop_details.loc[
            stop_details['stop_id'] == stop_id].loc[
                stop_details['direction'] == direction]

        # sort by arrival time
        df = df.sort_values('arrival_time')

        arrivals = pd.to_timedelta(df['arrival_time'])
        previous_arrivals = np.roll(arrivals, shift=1)

        # Calculate headways
        df['headway'] =  arrivals - previous_arrivals

        # Remove the first headway (no previous arrival time to compare with)
        df.iloc[0, -1] = None

        df_headways = pd.concat([df_headways, df])


    return df_headways


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,headway
20066,6530032245020,00:19:38,00:19:38,10517,23,Museum of Science & Industry,0,19048,0,0,55,65315,East,NaT
18430,6530029385010,00:46:42,00:46:42,10517,23,Museum of Science & Industry,0,19048,0,0,55,65315,East,0 days 00:27:04
4453,6530005916010,01:14:42,01:14:42,10517,23,Museum of Science & Industry,0,19048,1,1,55,65315,East,0 days 00:28:00
20786,6530034766020,01:44:42,01:44:42,10517,23,Museum of Science & Industry,0,19048,1,1,55,65315,East,0 days 00:30:00
14211,6530023460010,01:48:12,01:48:12,10517,10,Ashland,0,6677,1,1,55,65315,East,0 days 00:03:30
24298,6530039964020,02:14:42,02:14:42,10517,23,Museum of Science & Industry,0,19048,2,2,55,65315,East,0 days 00:26:30
20910,6530034887010,02:15:12,02:15:12,10517,10,Ashland,0,6677,2,2,55,65315,East,0 days 00:00:30
172,6530000232020,02:44:42,02:44:42,10517,10,Museum of Science & Industry,0,6677,2,2,55,65315,East,0 days 00:29:30
19819,6530031965020,03:14:17,03:14:17,10517,10,Museum of Science & Industry,0,6677,3,3,55,65301,East,0 days 00:29:35
21261,6530034445020,03:42:47,03:42:47,10517,10,Ashland,0,6677,3,3,55,65301,East,0 days 00:28:30


In [ ]:

# For a given stop, find the service ids that use it.

stop_id = '14122'

# filter stop times to a single stop:
stop_times = stop_times.loc[stop_times['stop_id'] == stop_id]

# list service ids that use this stop
service_ids = stop_times['service_id'].unique().tolist()

service_ids


In [ ]:
# Some services run at overlapping times on the same route. Each unique
# combination of services running simultaneously (or single service where there is only one) 
# will be considered together in a timeframe for the purpose of calculating headway stats.  
# Find the times when any service at this stop and route is starting or ending in each direction
# of travel, and use those times to identify time ranges.


# set up a dataframe to hold time ranges
df_timeranges = pd.DataFrame()

directions = stop_times_service['direction'].unique().tolist()

# split by direction as a precaution - not sure if there acutally are
# any stops that serve multiple directions on the same route, but it's possible.
for direction in directions:

    # set to contain all unique service start and end times
    start_end_times = set()

    # find the start and end times for each service running for this route, stop, and direction
    for service_id in service_ids:

        # filter stop times to the ones on this service and direction
        stop_times_service = stop_times_single_stop.loc[
            stop_times_single_stop['stop_id'] == stop_id].loc[
                stop_times_single_stop['service_id'] == service_id].loc[
                    stop_times_single_stop['direction'] == direction]

        # find start and end times for this service
        service_start = min(stop_times_service['arrival_time'])
        service_end = max(stop_times_service['arrival_time'])

        # add start/end times for this servide to the set of all start/end times
        start_end_times.add(service_start)
        start_end_times.add(service_end)

    # sort the start/end times chronologically
    start_end_times = list(start_end_times)
    start_end_times.sort()

    # set up a dataframe to contain the time ranges
    df_timeranges_direction = pd.DataFrame()

    # start time for each range. Don't include the last one
    # since there is no available end time.
    df_timeranges_direction ['start_time'] = start_end_times[:-1]

    # end time for each range (start of the next range). Don't
    # include the last one since there is no available end time.
    df_timeranges_direction['end_time'] = np.roll(start_end_times,shift=-1)[:-1]

    # add the direction to the dataframe
    df_timeranges_direction['direction'] = direction

    # add time range data for this direction to the overall time range dataframe
    df_timeranges = pd.concat([df_timeranges, df_timeranges_direction])

df_timeranges


In [36]:
trip_summary_test = trip_summary.loc[trip_summary['service_id'] == service_id]
service_start = min(trip_summary_test['stop_time'])
service_end = max(trip_summary_test['stop_time'])

print(f'Service id {service_id} starts at {service_start} and ends at {service_end}.')



Service id 65301 starts at 2023-01-09 03:10:00+00:00 and ends at 2023-01-10 00:03:30+00:00.


In [24]:



df_output = pd.DataFrame()

for stop_id, service_id in set(zip(trip_summary['stop_id'], trip_summary['service_id'])):

            # filter data
            trip_summary_stop_service = trip_summary.loc[
                    (trip_summary['stop_id'] == stop_id) 
                    & (trip_summary['service_id'] == service_id)
                    ]

            # Sort chronologically
            trip_summary_stop_service.sort_values(by='stop_time',ascending=True, inplace=True)

            # list stop times in chronological order
            stop_times = trip_summary_stop_service['stop_time'].tolist()

            # calculate previous stop times
            prev_stop_times = np.roll(stop_times,1)
            trip_summary_stop_service['previous_stop_time'] = prev_stop_times

            # calculate headway
            trip_summary_stop_service['scheduled_headway'] = (
                    trip_summary_stop_service['stop_time'] - trip_summary_stop_service['previous_stop_time']
                    )

            # drop previous stop time column, no longer needed
            trip_summary_stop_service = trip_summary_stop_service.drop('previous_stop_time', axis=1)

            # Remove headway from the first bus in the dataset since we don't have the 
            # previous bus to compare with
            trip_summary_stop_service['scheduled_headway'].iloc[0] = None
            
            # Add
            df_output = pd.concat([df_output, trip_summary_stop_service])



/var/folders/f2/c6nyrb_57g50fl1tj29bgpq40000gn/T/ipykernel_48651/3090683415.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_summary_stop_service.sort_values(by='stop_time',ascending=True, inplace=True)
/var/folders/f2/c6nyrb_57g50fl1tj29bgpq40000gn/T/ipykernel_48651/3090683415.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_summary_stop_service['previous_stop_time'] = prev_stop_times
/var/folders/f2/c6nyrb_57g50fl1tj29bgpq40000gn/T/ipykernel_48651/3090683415.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
df_output

## Try it out: Get Scheduled Stop Times and Headways

In [26]:
%%capture --no-display 
# turn off warnings

# Test:  Geet headways for the 55-Garfield bus on 1/9/2023 using the default times (2:30am 1/9 through 2:30am 1/10)
scheduled_headways = get_scheduled_headways('55', '2023-01-09')

In [48]:
### TODO:

# Note that some buses have very long scheduled headways.  Example below - Route 55, stop 10495.
# 55th and Kilpatrick eastbound.  Does this make sense?  It 

scheduled_headways.loc[['stop_id'] == '10495'].sort_values(['stop_id', 'arrival_time']).head(50)

KeyError: 'False: boolean label can not be used without a boolean index'

In [42]:
# Other stops do seem to make sense.  Example: Stop 10603, Garfield and Ashland WB
# on route 55:

scheduled_headways.loc[scheduled_headways['stop_id'] == '10603'].sort_values(['stop_id', 'arrival_time'])


,route_id,service_id,trip_id,direction_id,block_id,shape_id,direction,wheelchair_accessible,schd_trip_id,raw_date,...,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,scheduled_headway
985477,55,65301,6530028875020,0,653000003137,65301295,West,1,28875020,2023-01-09 00:00:00+00:00,...,2023-01-09 03:28:00+00:00,03:28:00,10603,1,St Louis,0,0,3,3,NaT
987031,55,65301,6530033660020,0,653000003140,65301295,West,1,33660020,2023-01-09 00:00:00+00:00,...,2023-01-09 03:55:30+00:00,03:55:30,10603,1,St Louis,0,0,3,3,0 days 00:27:30
986920,55,65301,6530033568020,0,653000003141,65301295,West,1,33568020,2023-01-09 00:00:00+00:00,...,2023-01-09 04:18:00+00:00,04:18:00,10603,1,St Louis,0,0,4,4,0 days 00:22:30
975487,55,65301,6530000010020,0,653000003086,65301295,West,1,10020,2023-01-09 00:00:00+00:00,...,2023-01-09 04:37:30+00:00,04:37:30,10603,1,St Louis,0,0,4,4,0 days 00:19:30
983665,55,65301,6530023086020,0,653000003140,65305425,West,1,23086020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:00:37+00:00,05:00:37,10603,35,Midway Orange Line,0,25705,5,5,0 days 00:23:07
976116,55,65301,6530001696020,0,653000003152,65301295,West,1,1696020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:11:30+00:00,05:11:30,10603,1,St Louis,0,0,5,5,0 days 00:10:53
985218,55,65301,6530027998020,0,653000002970,65301295,West,1,27998020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:35:30+00:00,05:35:30,10603,1,St Louis,0,0,5,5,0 days 00:24:00
986476,55,65301,6530032163020,0,653000002965,65301295,West,1,32163020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:57:00+00:00,05:57:00,10603,1,St Louis,0,0,5,5,0 days 00:21:30
988215,55,65301,6530038932020,0,653000003090,65301295,West,1,38932020,2023-01-09 00:00:00+00:00,...,2023-01-09 06:23:00+00:00,06:23:00,10603,1,St Louis,0,0,6,6,0 days 00:26:00
983849,55,65301,6530023418020,0,653000002878,65301295,West,1,23418020,2023-01-09 00:00:00+00:00,...,2023-01-09 06:42:30+00:00,06:42:30,10603,1,St Louis,0,0,6,6,0 days 00:19:30


In [39]:
# Get scheduled headway stats
scheduled_headway_stats = get_headway_stats(scheduled_headways, 'scheduled_headway')

scheduled_headway_stats 

{'mean': Timedelta('0 days 01:33:54.296969696'),
 'max': Timedelta('0 days 16:59:57'),
 'min': Timedelta('0 days 00:01:00'),
 '25th_percentile': Timedelta('0 days 00:12:00'),
 'median': Timedelta('0 days 00:18:00'),
 '75th_percentile': Timedelta('0 days 01:00:00')}

# Get acutal and scheduled headway stats for every stop on a route


In [90]:
# Latest GTFS feed date available from transitfeeds.com
gtfs_version_id = '20230105'

def get_stats_for_stops(
    route_id:str, 
    service_date_string:str, 
    start_timedelta_string:str='02:30', 
    end_timedelta_string:str='26:30') -> pd.DataFrame:

    def get_headway_stats_by_stop(headway_dataframe:pd.DataFrame, stop_id_column:str, headway_column:str) -> pd.DataFrame:

        df_output = pd.DataFrame()

        # df_output['stop_ids'] = scheduled_headways['stop_id'].groupby(by='stop_id')
        df_output[f'{headway_column}_mean'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].mean()
        df_output[f'{headway_column}_min'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].min()
        df_output[f'{headway_column}_25th_percentile'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].quantile(0.25)
        df_output[f'{headway_column}_median'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].median()
        df_output[f'{headway_column}_75th_percentile'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].quantile(.75)
        df_output[f'{headway_column}_max'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].max()

        # move stop id from index to a regular column named 'stop_id'
        df_output.reset_index(drop=True)
        df_output['stop_id'] = df_output.index
        return df_output

    # Scheduled data from gtfs
    gtfs_feed = download_extract_format(gtfs_version_id)

    scheduled_headways = get_scheduled_headways(
        route_id, 
        service_date_string, 
        start_timedelta_string, 
        end_timedelta_string
        )

    df_scheduled_headway_stops = get_headway_stats_by_stop(scheduled_headways,'stop_id', 'scheduled_headway')

    return df_scheduled_headway_stops

    # # Actual data from cta API
    # vehicles = get_chn_vehicles(service_date_string,start_timedelta_string, end_timedelta_string)

    # actual_headways = get_headways(route_id, vehicles)

    # df_actual_headway_stops = get_headway_stats_by_stop(actual_headways, 'stpid', 'est_headway')

    # # Merge scheduled and actual stats
    # output = df_scheduled_headway_stops.merge(df_actual_headway_stops, on='stop_id', how='outer')

    # return output



In [91]:
%%capture --no-display

get_stats_for_stops('55','2023-01-09')

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
INFO:root:stops.txt loaded
INFO:root:stop_times.txt loaded
INFO:root:routes.txt loaded
INFO:root:trips.txt loaded
INFO:root:calendar.txt loaded
INFO:root:calendar_dates.txt loaded
INFO:root:shapes.txt loaded


,scheduled_headway_mean,scheduled_headway_min,scheduled_headway_25th_percentile,scheduled_headway_median,scheduled_headway_75th_percentile,scheduled_headway_max,stop_id
stop_id,,,,,,,
10495,0 days 02:00:09.833333333,0 days 00:57:00,0 days 00:58:11.250000,0 days 01:02:15,0 days 01:02:26.250000,0 days 04:58:45,10495
10501,NaT,NaT,NaT,NaT,NaT,NaT,10501
10502,NaT,NaT,NaT,NaT,NaT,NaT,10502
10505,NaT,NaT,NaT,NaT,NaT,NaT,10505
10507,0 days 00:52:14.444444444,0 days 00:01:00,0 days 00:18:00,0 days 00:21:31,0 days 00:27:17,0 days 07:58:04,10507
...,...,...,...,...,...,...,...
17563,NaT,NaT,NaT,NaT,NaT,NaT,17563
18018,NaT,NaT,NaT,NaT,NaT,NaT,18018
18032,0 days 05:29:54.500000,0 days 05:00:39,-80064 days +01:24:42.108917760,0 days 05:00:39,-26688 days +03:48:40.036306432,0 days 05:59:10,18032


In [63]:
%%capture --no-display
# turn off warnings

# Scratch
gtfs_version_id = '20230105'
route_id = '55'
service_date_string = '2023-01-09'
start_timedelta_string ='02:30'
end_timedelta_string = '26:30'

gtfs_feed = download_extract_format(gtfs_version_id)



# scheduled_headways = get_scheduled_headways(
#     route_id, 
#     service_date_string, 
#     start_timedelta_string, 
#     end_timedelta_string
#     )


INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
INFO:root:stops.txt loaded
INFO:root:stop_times.txt loaded
INFO:root:routes.txt loaded
INFO:root:trips.txt loaded
INFO:root:calendar.txt loaded
INFO:root:calendar_dates.txt loaded
INFO:root:shapes.txt loaded


In [62]:

df_output = pd.DataFrame()

# df_output['stop_ids'] = scheduled_headways['stop_id'].groupby(by='stop_id')
df_output['sched_mean'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].mean()
df_output['sched_min'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].min()
df_output['sched_25th_percentile'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].quantile(0.25)
df_output['sched_median'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].median()
df_output['sched_75th_percentile'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].quantile(.75)
df_output['sched_max'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].max()

df_output




    

,sched_mean,sched_min,sched_25th_percentile,sched_median,sched_75th_percentile,sched_max
stop_id,,,,,,
10495,0 days 02:00:09.833333333,0 days 00:57:00,0 days 00:58:11.250000,0 days 01:02:15,0 days 01:02:26.250000,0 days 04:58:45
10501,NaT,NaT,NaT,NaT,NaT,NaT
10502,NaT,NaT,NaT,NaT,NaT,NaT
10505,NaT,NaT,NaT,NaT,NaT,NaT
10507,0 days 00:52:14.444444444,0 days 00:01:00,0 days 00:18:00,0 days 00:21:31,0 days 00:27:17,0 days 07:58:04
...,...,...,...,...,...,...
17563,NaT,NaT,NaT,NaT,NaT,NaT
18018,NaT,NaT,NaT,NaT,NaT,NaT
18032,0 days 05:29:54.500000,0 days 05:00:39,-80064 days +01:24:42.108917760,0 days 05:00:39,-26688 days +03:48:40.036306432,0 days 05:59:10


In [54]:
scheduled_headways.groupby(by='stop_id')['scheduled_headway'].mean()


quantile(0.25)

stop_id
10495   0 days 02:00:09.833333333
10501                         NaT
10502                         NaT
10505                         NaT
10507   0 days 00:52:14.444444444
                   ...           
17563                         NaT
18018                         NaT
18032      0 days 05:29:54.500000
18448                         NaT
6524              0 days 07:59:52
Name: scheduled_headway, Length: 98, dtype: timedelta64[ns]

In [14]:
### ChatGPT code:

import pandas as pd
import requests

# URL for the GTFS data
url = "http://www.transitchicago.com/downloads/sch_data/google_transit.zip"

# Download the data
response = requests.get(url)
open("google_transit.zip", "wb").write(response.content)

# Load the data into pandas dataframes
dataframes = {
    "routes": pd.read_csv("google_transit/routes.txt"),
    "trips": pd.read_csv("google_transit/trips.txt"),
    "stop_times": pd.read_csv("google_transit/stop_times.txt"),
    "stops": pd.read_csv("google_transit/stops.txt"),
    "calendar": pd.read_csv("google_transit/calendar.txt"),
}

# Print the first 5 rows of the "routes" dataframe
print(dataframes["routes"].head())


FileNotFoundError: [Errno 2] No such file or directory: 'google_transit/routes.txt'

In [16]:
# ChatGPT code
 
import pandas as pd

# Load the GTFS data into a pandas dataframe
stop_times = gtfs_feed.stop_times

# My edit:  convert to timedelta so the code can run
stop_times['arrival_time']=stop_times['arrival_time'].apply(pd.to_timedelta)

# Calculate the time difference between consecutive scheduled arrival times for each trip
stop_times["headway_secs"] = stop_times.sort_values(["trip_id", "stop_sequence"]).groupby("trip_id")["arrival_time"].diff().dt.total_seconds()

# Calculate the average headway for each bus stop
headways = stop_times.groupby("stop_id")["headway_secs"].mean()

# Print the headways in minutes
print(headways / 60)


stop_id
1          0.699632
10         0.379508
100        0.580469
1000       0.397299
10000     -0.097356
            ...    
9996       0.450604
9998       0.356522
9999       0.085172
999990    13.500000
999991     8.500000
Name: headway_secs, Length: 11062, dtype: float64
